In [ ]:
!pip install geopandas --upgrade
!pip install rasterio --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 16.7 MB 334 kB/s 
     |████████████████████████████████| 6.3 MB 56.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 5.4 MB/s 


In [ ]:
import geopandas as gpd
import rasterio
from google.colab import drive
import os
import pandas as pd
import numpy as np
os.environ['OPENCV_IO_ENABLE_JASPER'] = 'true'
import cv2 as cv
import matplotlib.pyplot as plt
from rasterio.features import rasterize
import math

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_list = []
band_unique_list = []
main_dir = '/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/sentinel-2-image'

img_df = pd.DataFrame(({'year': [], 'month': [], 'day': [], 'band': [], 'path': []}))

for i in os.listdir(main_dir)[:2]:

  for j in os.listdir(f'{main_dir}/{i}'):

    for file in os.listdir(f'{main_dir}/{i}/{j}/IMG_DATA'):
      
      if file.endswith('.jp2'):

        img_list.append(f'{main_dir}/{i}/{j}/IMG_DATA/{file}')
        if file[-7:-4] not in band_unique_list:
          band_unique_list.append(file[-7:-4])

        img_df.loc[len(img_df)] = [file[-16:-8][:4], file[-16:-8][4:6], file[-16:-8][6:], file[-7:-4], f'{main_dir}/{i}/{j}/IMG_DATA/{file}']

img_df

,year,month,day,band,path
0,2020,01,22,AOT,/content/drive/Shareddrives/arv_msg/Varuna Hac...
1,2020,01,22,B01,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2,2020,01,22,B02,/content/drive/Shareddrives/arv_msg/Varuna Hac...
3,2020,01,22,B03,/content/drive/Shareddrives/arv_msg/Varuna Hac...
4,2020,01,22,B04,/content/drive/Shareddrives/arv_msg/Varuna Hac...
...,...,...,...,...,...
2125,2021,12,22,B12,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2126,2021,12,22,B11,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2127,2021,12,22,SCL,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2128,2021,12,22,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...


In [ ]:
# separate tci

img_df_tci = img_df.loc[img_df.band == 'TCI'].copy()
img_df_tci

,year,month,day,band,path
13,2020,01,22,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
28,2020,02,11,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
43,2020,01,12,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
58,2020,01,02,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
73,2020,02,06,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
...,...,...,...,...,...
2069,2021,10,28,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2083,2021,11,27,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2098,2021,11,17,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...
2113,2021,12,27,TCI,/content/drive/Shareddrives/arv_msg/Varuna Hac...


In [ ]:
# separate tci with cloud index

img_df_tci_cloud = img_df.loc[img_df.band == 'TCI'].copy()
img_df_tci_cloud['cloud_index'] = img_df_tci_cloud.apply(lambda _: '', axis=1)

for i, row in img_df_tci_cloud.iterrows():
  img = cv.imread(row['path'], cv.IMREAD_GRAYSCALE)
  ret, outs = cv.threshold(src = img, thresh = 245, maxval = 255, type = cv.THRESH_BINARY)
  row['cloud_index'] = np.sum(outs == 255)/(2051*2051)*100

img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].shape

(48, 6)

In [ ]:
# target image

img_profile = rasterio.open('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/sentinel-2-image/2021/20210101/IMG_DATA/47PQS_20210101_B04.jp2').profile
label = gpd.read_file('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/training_area/traindata.shp').to_crs(img_profile['crs'])

target_image = rasterize(
  shapes=[(row.geometry, int(row.crop_type)) for i, row in label.iterrows()], 
  out_shape=(img_profile['width'], img_profile['height']),
  transform=img_profile['transform']
)
classes, pixel_counts = np.unique(target_image, return_counts=True)
print('Class pixel count:' + str(dict(zip(classes[1:], pixel_counts[1:]))))

print(target_image.shape)

Class pixel count:{1: 100162, 2: 29069, 3: 84342, 4: 23309}
(2051, 2051)


# ndvi, ndwi, gci, savi

In [ ]:
def indexstack(green, red, nir):
    
    def NDVIcalc(red, nir): 
        ndvi = (nir - red) / (nir + red) 
        return ndvi
    
    def NDWIcalc(green, nir):
        ndwi = (green - nir) / (green + nir)
        return ndwi
    
    def GCIcalc(green, nir):
        gci = (nir / green) - 1
        return gci

    def SAVIcalc(red, nir):
        savi = ((nir - red) / (nir + red + 0.5)) * (1 + 0.5)
        return savi
    
    ndvi = NDVIcalc(red, nir) 

    ndwi = NDWIcalc(green, nir)

    gci = GCIcalc(green, nir)
    
    savi = SAVIcalc(red, nir)

    ndvi = ndvi.transpose(1,2,0)
    ndwi = ndwi.transpose(1,2,0)
    gci = gci.transpose(1,2,0)
    savi = savi.transpose(1,2,0)

    index_stack = np.dstack((ndvi, ndwi, gci, savi))
    
    return index_stack

In [ ]:
# all 142

img_profile = rasterio.open('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/sentinel-2-image/2021/20210101/IMG_DATA/47PQS_20210101_B04.jp2').profile
label = gpd.read_file('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/training_area/traindata.shp').to_crs(img_profile['crs'])

target_image = rasterize(
  shapes=[(row.geometry, int(row.crop_type)) for i, row in label.iterrows()], 
  out_shape=(img_profile['width'], img_profile['height']),
  transform=img_profile['transform']
)

out_df = pd.DataFrame({'date':[], 'ndvi': [], 'ndwi':[], 'gci':[], 'savi':[], 'label':[]})

for i in range(142):


  outs_means_1 = np.zeros((1,4), dtype = 'float64')
  outs_means_2 = np.zeros((1,4), dtype = 'float64')
  outs_means_3 = np.zeros((1,4), dtype = 'float64')
  outs_means_4 = np.zeros((1,4), dtype = 'float64')

  c1 = 0
  c2 = 0
  c3 = 0
  c4 = 0

  tci_path = img_df_tci.iloc[i].path

  green_path = f'{tci_path[:len(tci_path)-7]}B03.jp2'
  red_path = f'{tci_path[:len(tci_path)-7]}B04.jp2'
  nir_path = f'{tci_path[:len(tci_path)-7]}B08.jp2'

  img_green = rasterio.open(green_path)
  img_red = rasterio.open(red_path)
  img_nir = rasterio.open(nir_path)

  img_green = img_green.read()
  img_red = img_red.read()
  img_nir = img_nir.read()

  outs_noclass = indexstack(img_green, img_red, img_nir)

  for row in range(2051):
    for col in range(2051):

      if target_image[row][col] == 1:
        outs_means_1 = outs_means_1 + outs_noclass[row][col]
        c1+=1
      if target_image[row][col] == 2:
        outs_means_2 = outs_means_2 + outs_noclass[row][col]
        c2+=1
      if target_image[row][col] == 3:
        outs_means_3 = outs_means_3 + outs_noclass[row][col]
        c3+=1
      if target_image[row][col] == 4:
        outs_means_4 = outs_means_4 + outs_noclass[row][col]
        c4+=1

  outs_means_1 = outs_means_1 / c1
  outs_means_2 = outs_means_2 / c2
  outs_means_3 = outs_means_3 / c3
  outs_means_4 = outs_means_4 / c4

  date = f'{img_df_tci.iloc[i].year}-{img_df_tci.iloc[i].month}-{img_df_tci.iloc[i].day}'
  
  outs_means_1 = np.append([date], outs_means_1)
  outs_means_1 = np.append(outs_means_1, [1])
  outs_means_2 = np.append([date], outs_means_2)
  outs_means_2 = np.append(outs_means_2, [2])
  outs_means_3 = np.append([date], outs_means_3)
  outs_means_3 = np.append(outs_means_3, [3])
  outs_means_4 = np.append([date], outs_means_4)
  outs_means_4 = np.append(outs_means_4, [4])

  out_df.loc[len(out_df)] = outs_means_1
  out_df.loc[len(out_df)] = outs_means_2
  out_df.loc[len(out_df)] = outs_means_3
  out_df.loc[len(out_df)] = outs_means_4

  out_df.to_csv('/content/drive/Shareddrives/arv_msg/colab_notebooks/data.csv')
  print(f'File {i}, succesful!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
File 5, succesful!
File 6, succesful!
File 7, succesful!
File 8, succesful!
File 9, succesful!
File 10, succesful!
File 11, succesful!
File 12, succesful!
File 13, succesful!
File 14, succesful!
File 15, succesful!
File 16, succesful!
File 17, succesful!
File 18, succesful!
File 19, succesful!
File 20, succesful!
File 21, succesful!
File 22, succesful!
File 23, succesful!
File 24, succesful!
File 25, succesful!
File 26, succesful!
File 27, succesful!
File 28, succesful!
File 29, succesful!
File 30, succesful!
File 31, succesful!
File 32, succesful!
File 33, succesful!
File 34, succesful!
File 35, succesful!
File 36, succesful!
File 37, succesful!
File 38, succesful!
File 39, succesful!
File 40, succesful!
File 41, succesful!
File 42, succesful!
File 43, succesful!
File 44, succesful!
File 45, succesful!
File 46, succesful!
File 47, succesful!
File 48, succesful!
File 49, succesful!
File 50, s

In [ ]:
# no cloud

img_profile = rasterio.open('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/sentinel-2-image/2021/20210101/IMG_DATA/47PQS_20210101_B04.jp2').profile
label = gpd.read_file('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/training_area/traindata.shp').to_crs(img_profile['crs'])

target_image = rasterize(
  shapes=[(row.geometry, int(row.crop_type)) for i, row in label.iterrows()], 
  out_shape=(img_profile['width'], img_profile['height']),
  transform=img_profile['transform']
)

out_df = pd.DataFrame({'date':[], 'ndvi': [], 'ndwi':[], 'gci':[], 'savi':[], 'label':[]})

for i in range(48):


  outs_means_1 = np.zeros((1,4), dtype = 'float64')
  outs_means_2 = np.zeros((1,4), dtype = 'float64')
  outs_means_3 = np.zeros((1,4), dtype = 'float64')
  outs_means_4 = np.zeros((1,4), dtype = 'float64')

  c1 = 0
  c2 = 0
  c3 = 0
  c4 = 0

  tci_path = img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].path

  green_path = f'{tci_path[:len(tci_path)-7]}B03.jp2'
  red_path = f'{tci_path[:len(tci_path)-7]}B04.jp2'
  nir_path = f'{tci_path[:len(tci_path)-7]}B08.jp2'

  img_green = rasterio.open(green_path)
  img_red = rasterio.open(red_path)
  img_nir = rasterio.open(nir_path)

  img_green = img_green.read()
  img_red = img_red.read()
  img_nir = img_nir.read()

  outs_noclass = indexstack(img_green, img_red, img_nir)

  for row in range(2051):
    for col in range(2051):

      if target_image[row][col] == 1:
        outs_means_1 = outs_means_1 + outs_noclass[row][col]
        c1+=1
      if target_image[row][col] == 2:
        outs_means_2 = outs_means_2 + outs_noclass[row][col]
        c2+=1
      if target_image[row][col] == 3:
        outs_means_3 = outs_means_3 + outs_noclass[row][col]
        c3+=1
      if target_image[row][col] == 4:
        outs_means_4 = outs_means_4 + outs_noclass[row][col]
        c4+=1

  outs_means_1 = outs_means_1 / c1
  outs_means_2 = outs_means_2 / c2
  outs_means_3 = outs_means_3 / c3
  outs_means_4 = outs_means_4 / c4

  date = f'{img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].year}-{img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].month}-{img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].day}'
  
  outs_means_1 = np.append([date], outs_means_1)
  outs_means_1 = np.append(outs_means_1, [1])
  outs_means_2 = np.append([date], outs_means_2)
  outs_means_2 = np.append(outs_means_2, [2])
  outs_means_3 = np.append([date], outs_means_3)
  outs_means_3 = np.append(outs_means_3, [3])
  outs_means_4 = np.append([date], outs_means_4)
  outs_means_4 = np.append(outs_means_4, [4])

  out_df.loc[len(out_df)] = outs_means_1
  out_df.loc[len(out_df)] = outs_means_2
  out_df.loc[len(out_df)] = outs_means_3
  out_df.loc[len(out_df)] = outs_means_4

  out_df.to_csv('/content/drive/Shareddrives/arv_msg/colab_notebooks/data_nocloud.csv')
  print(f'File {i}, succesful!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
File 5, succesful!
File 6, succesful!
File 7, succesful!
File 8, succesful!
File 9, succesful!
File 10, succesful!
File 11, succesful!
File 12, succesful!
File 13, succesful!
File 14, succesful!
File 15, succesful!
File 16, succesful!
File 17, succesful!
File 18, succesful!
File 19, succesful!
File 20, succesful!
File 21, succesful!
File 22, succesful!
File 23, succesful!
File 24, succesful!
File 25, succesful!
File 26, succesful!
File 27, succesful!
File 28, succesful!
File 29, succesful!
File 30, succesful!
File 31, succesful!
File 32, succesful!
File 33, succesful!
File 34, succesful!
File 35, succesful!
File 36, succesful!
File 37, succesful!
File 38, succesful!
File 39, succesful!
File 40, succesful!
File 41, succesful!
File 42, succesful!
File 43, succesful!
File 44, succesful!
File 45, succesful!
File 46, succesful!
File 47, succesful!


# evi, vari, ndre, gndvi

In [ ]:
def indexstack_2(blue, green, red, red_edge, nir):
    
    def EVIcalc(blue, red, nir): 
        evi = 2.5 * (nir - red) / ((nir + 6.0 * red - 7.5 * blue) + 1.0)
        return evi
    
    def VARIcalc(blue, green, red):
        vari = (green - red) / (green + red - blue)
        return vari
    
    def NDREcalc(red_edge, nir):
        ndre = (nir - red_edge) / (nir + red_edge)
        return ndre

    def GNDVIcalc(green, nir):
        gndvi = (nir - green) / (nir + green)
        return gndvi
    
    evi = EVIcalc(blue, red, nir) 

    vari = VARIcalc(blue, green, red)

    ndre = NDREcalc(red_edge, nir)
    
    gndvi = GNDVIcalc(green, nir)

    evi = evi.transpose(1,2,0)
    vari = vari.transpose(1,2,0)
    ndre = ndre.transpose(1,2,0)
    gndvi = gndvi.transpose(1,2,0)

    index_stack_2 = np.dstack((evi, vari, ndre, gndvi))
    
    return index_stack_2

In [ ]:
def raster_resize(src):
    w = src.read(1, window=rasterio.windows.Window(0, 0, 2051, 2051))
    profile = src.profile
    profile['width'] = 2051
    profile['height'] = 2051
    result = np.full((2051,2051), dtype=profile['dtype'], fill_value=profile['nodata'])
    return result

In [ ]:
# no cloud

img_profile = rasterio.open('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/sentinel-2-image/2021/20210101/IMG_DATA/47PQS_20210101_B04.jp2').profile
label = gpd.read_file('/content/drive/Shareddrives/arv_msg/Varuna Hackathon 2022/training_area/traindata.shp').to_crs(img_profile['crs'])

target_image = rasterize(
  shapes=[(row.geometry, int(row.crop_type)) for i, row in label.iterrows()], 
  out_shape=(img_profile['width'], img_profile['height']),
  transform=img_profile['transform']
)

out_df = pd.DataFrame({'date':[], 'evi': [], 'vari':[], 'ndre':[], 'gndvi':[], 'label':[]})

for i in range(48):


  outs_means_1 = np.zeros((1,4), dtype = 'float64')
  outs_means_2 = np.zeros((1,4), dtype = 'float64')
  outs_means_3 = np.zeros((1,4), dtype = 'float64')
  outs_means_4 = np.zeros((1,4), dtype = 'float64')

  c1 = 0
  c2 = 0
  c3 = 0
  c4 = 0

  tci_path = img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].path

  blue_path = f'{tci_path[:len(tci_path)-7]}B02.jp2'
  green_path = f'{tci_path[:len(tci_path)-7]}B03.jp2'
  red_path = f'{tci_path[:len(tci_path)-7]}B04.jp2'
  red_edge_path = f'{tci_path[:len(tci_path)-7]}B06.jp2'
  nir_path = f'{tci_path[:len(tci_path)-7]}B08.jp2'

  img_blue = rasterio.open(blue_path)
  img_green = rasterio.open(green_path)
  img_red = rasterio.open(red_path)
  img_red_edge = rasterio.open(red_edge_path)
  img_nir = rasterio.open(nir_path)

  img_blue = img_blue.read()
  img_green = img_green.read()
  img_red = img_red.read()
  img_red_edge = raster_resize(img_red_edge)
  img_nir = img_nir.read()

  outs_noclass = indexstack_2(img_blue, img_green, img_red, img_red_edge, img_nir)

  for row in range(2051):
    for col in range(2051):

      if target_image[row][col] == 1:
        outs_means_1 = outs_means_1 + outs_noclass[row][col]
        c1+=1
      if target_image[row][col] == 2:
        outs_means_2 = outs_means_2 + outs_noclass[row][col]
        c2+=1
      if target_image[row][col] == 3:
        outs_means_3 = outs_means_3 + outs_noclass[row][col]
        c3+=1
      if target_image[row][col] == 4:
        outs_means_4 = outs_means_4 + outs_noclass[row][col]
        c4+=1

  outs_means_1 = outs_means_1 / c1
  outs_means_2 = outs_means_2 / c2
  outs_means_3 = outs_means_3 / c3
  outs_means_4 = outs_means_4 / c4

  date = f'{img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].year}-{img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].month}-{img_df_tci_cloud.loc[img_df_tci_cloud.cloud_index <= 1.5].iloc[i].day}'
  
  outs_means_1 = np.append([date], outs_means_1)
  outs_means_1 = np.append(outs_means_1, [1])
  outs_means_2 = np.append([date], outs_means_2)
  outs_means_2 = np.append(outs_means_2, [2])
  outs_means_3 = np.append([date], outs_means_3)
  outs_means_3 = np.append(outs_means_3, [3])
  outs_means_4 = np.append([date], outs_means_4)
  outs_means_4 = np.append(outs_means_4, [4])

  out_df.loc[len(out_df)] = outs_means_1
  out_df.loc[len(out_df)] = outs_means_2
  out_df.loc[len(out_df)] = outs_means_3
  out_df.loc[len(out_df)] = outs_means_4

  out_df.to_csv('/content/drive/Shareddrives/arv_msg/colab_notebooks/data_2_nocloud.csv')
  print(f'File {i}, succesful!')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
File 5, succesful!
File 6, succesful!
File 7, succesful!
File 8, succesful!
File 9, succesful!
File 10, succesful!
File 11, succesful!
File 12, succesful!
File 13, succesful!
File 14, succesful!
File 15, succesful!
File 16, succesful!
File 17, succesful!
File 18, succesful!
File 19, succesful!
File 20, succesful!
File 21, succesful!
File 22, succesful!
File 23, succesful!
File 24, succesful!
File 25, succesful!
File 26, succesful!
File 27, succesful!
File 28, succesful!
File 29, succesful!
File 30, succesful!
File 31, succesful!
File 32, succesful!
File 33, succesful!
File 34, succesful!
File 35, succesful!
File 36, succesful!
File 37, succesful!
File 38, succesful!
File 39, succesful!
File 40, succesful!
File 41, succesful!
File 42, succesful!
File 43, succesful!
File 44, succesful!
File 45, succesful!
File 46, succesful!
File 47, succesful!


# Test Section

In [39]:
os.listdir('/content')

['.config',
 '47PQS_20210101_B02.jp2',
 '47PQS_20210401_B08.jp2',
 '47PQS_20210101_B08.jp2',
 '47PQS_20210501_B08.jp2',
 '47PQS_20210401_B03.jp2',
 '47PQS_20210401_B04.jp2',
 'drive',
 '47PQS_20210101_B04.jp2',
 '47PQS_20210205_B04.jp2',
 '47PQS_20210205_B03.jp2',
 '47PQS_20210205_B02.jp2',
 '47PQS_20210401_B02.jp2',
 '47PQS_20210302_B04.jp2',
 '.ipynb_checkpoints',
 'testing_area.zip',
 '47PQS_20210302_B03.jp2',
 'test_data.shp',
 '47PQS_20210302_B02.jp2',
 '47PQS_20200127_TCI.jp2',
 '47PQS_20210101_B03.jp2',
 '47PQS_20210501_B03.jp2',
 '47PQS_20210501_B04.jp2',
 '47PQS_20210205_B08.jp2',
 '47PQS_20210302_B08.jp2',
 '47PQS_20210501_B02.jp2',
 'sample_data']

In [44]:
main_dir = '/content'

img_df = pd.DataFrame(({'year': [], 'month': [], 'day': [], 'band': [], 'path': []}))

for file in os.listdir(main_dir):
  
  if file.endswith('.jp2'):

    if file[-7:-4] not in band_unique_list:
      band_unique_list.append(file[-7:-4])

    img_df.loc[len(img_df)] = [file[-16:-8][:4], file[-16:-8][4:6], file[-16:-8][6:], file[-7:-4], f'{main_dir}/{file}']

img_df

,year,month,day,band,path
0,2021,01,01,B02,/content/47PQS_20210101_B02.jp2
1,2021,04,01,B08,/content/47PQS_20210401_B08.jp2
2,2021,01,01,B08,/content/47PQS_20210101_B08.jp2
3,2021,05,01,B08,/content/47PQS_20210501_B08.jp2
4,2021,04,01,B03,/content/47PQS_20210401_B03.jp2
5,2021,04,01,B04,/content/47PQS_20210401_B04.jp2
6,2021,01,01,B04,/content/47PQS_20210101_B04.jp2
7,2021,02,05,B04,/content/47PQS_20210205_B04.jp2
8,2021,02,05,B03,/content/47PQS_20210205_B03.jp2
9,2021,02,05,B02,/content/47PQS_20210205_B02.jp2


In [41]:
def indexstack_3(blue, green, red, nir):

    def NDVIcalc(red, nir): 
        ndvi = (nir - red) / (nir + red) 
        return ndvi
    
    def NDWIcalc(green, nir):
        ndwi = (green - nir) / (green + nir)
        return ndwi
    
    def GCIcalc(green, nir):
        gci = (nir / green) - 1
        return gci

    def SAVIcalc(red, nir):
        savi = ((nir - red) / (nir + red + 0.5)) * (1 + 0.5)
        return savi


    def EVIcalc(blue, red, nir): 
        evi = 2.5 * (nir - red) / ((nir + 6.0 * red - 7.5 * blue) + 1.0)
        return evi
    
    def VARIcalc(blue, green, red):
        vari = (green - red) / (green + red - blue)
        return vari
    
    def GNDVIcalc(green, nir):
        gndvi = (nir - green) / (nir + green)
        return gndvi
    

    ndvi = NDVIcalc(red, nir) 

    ndwi = NDWIcalc(green, nir)

    gci = GCIcalc(green, nir)
    
    savi = SAVIcalc(red, nir)

    evi = EVIcalc(blue, red, nir) 

    vari = VARIcalc(blue, green, red)
    
    gndvi = GNDVIcalc(green, nir)

    ndvi = ndvi.transpose(1,2,0)
    ndwi = ndwi.transpose(1,2,0)
    gci = gci.transpose(1,2,0)
    savi = savi.transpose(1,2,0)
    evi = evi.transpose(1,2,0)
    vari = vari.transpose(1,2,0)
    gndvi = gndvi.transpose(1,2,0)

    index_stack_2 = np.dstack((ndvi, ndwi, gci, savi, evi, vari, gndvi))
    
    return index_stack_2

In [ ]:
# test no cloud

img_profile = rasterio.open('47PQS_20210101_B02.jp2').profile
label = gpd.read_file('/content/test_data.shp').to_crs(img_profile['crs'])

out_df = pd.DataFrame({'ndvi': [], 'ndwi':[], 'gci':[], 'savi':[], 'evi': [], 'vari':[], 'gndvi':[]})

for area in range(565):

  target_image = rasterize(
  shapes=[(label.iloc[area].geometry, 1)], 
  out_shape=(img_profile['width'], img_profile['height']),
  transform=img_profile['transform']
  )

  c=0

  for i in range(5):

    outs_means = np.zeros((1,7), dtype = 'float64')

    tci_path = img_df.iloc[i].path

    blue_path = f'{tci_path[:len(tci_path)-7]}B02.jp2'
    green_path = f'{tci_path[:len(tci_path)-7]}B03.jp2'
    red_path = f'{tci_path[:len(tci_path)-7]}B04.jp2'
    nir_path = f'{tci_path[:len(tci_path)-7]}B08.jp2'

    img_blue = rasterio.open(blue_path)
    img_green = rasterio.open(green_path)
    img_red = rasterio.open(red_path)
    img_nir = rasterio.open(nir_path)

    img_blue = img_blue.read()
    img_green = img_green.read()
    img_red = img_red.read()
    img_nir = img_nir.read()

    outs_noclass = indexstack_3(img_blue, img_green, img_red, img_nir)

    for row in range(2051):
      for col in range(2051):

        if target_image[row][col] == 1:
          outs_means = outs_means + outs_noclass[row][col]
          c+=1
    
    print(f'File {i}, succesful!')

  outs_means = outs_means / c * 5
  out_df.loc[len(out_df)] = outs_means[0]
  out_df.to_csv('/content/data_test.csv')
    
  
  print(f'Area {area}, succesful!/n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launc

File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 0, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 1, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 2, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 3, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 4, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 5, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 6, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesful!
File 4, succesful!
Area 7, succesful!/n
File 0, succesful!
File 1, succesful!
File 2, succesful!
File 3, succesf